# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 14 2023 8:30AM,257301,10,0086377489,ISDIN Corporation,Released
1,Feb 14 2023 8:30AM,257301,10,0086377509,ISDIN Corporation,Released
2,Feb 14 2023 8:30AM,257301,10,0086377375,ISDIN Corporation,Released
3,Feb 14 2023 8:30AM,257301,10,0086377430,ISDIN Corporation,Released
4,Feb 14 2023 8:30AM,257301,10,0086377432,ISDIN Corporation,Released
5,Feb 14 2023 8:30AM,257301,10,0086377431,ISDIN Corporation,Released
6,Feb 14 2023 8:30AM,257301,10,0086377487,ISDIN Corporation,Released
7,Feb 14 2023 8:30AM,257301,10,0086377488,ISDIN Corporation,Released
8,Feb 14 2023 8:30AM,257301,10,0086377111,ISDIN Corporation,Released
9,Feb 14 2023 8:30AM,257301,10,0086377120,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,257298,Released,50
37,257301,Released,33
38,257302,Released,6
39,257303,Released,2
40,257304,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257298,NaN,NaN,50.0
257301,NaN,NaN,33.0
257302,NaN,NaN,6.0
257303,NaN,NaN,2.0
257304,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257194,40.0,0.0,1.0
257198,0.0,1.0,0.0
257199,0.0,0.0,8.0
257200,0.0,0.0,10.0
257201,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257194,40,0,1
257198,0,1,0
257199,0,0,8
257200,0,0,10
257201,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257194,40,0,1
1,257198,0,1,0
2,257199,0,0,8
3,257200,0,0,10
4,257201,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257194,40,,1
1,257198,,1,
2,257199,,,8
3,257200,,,10
4,257201,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation
32,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation
80,Feb 14 2023 8:30AM,257297,10,ISDIN Corporation
86,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc."
94,Feb 14 2023 8:30AM,257303,10,"ClearSpec, LLC"
96,Feb 14 2023 8:30AM,257294,10,"Snap Medical Industries, LLC"
97,Feb 14 2023 8:30AM,257295,10,"Snap Medical Industries, LLC"
100,Feb 14 2023 8:30AM,257304,10,Acute Outpatient Solutions
101,Feb 13 2023 3:54PM,257293,19,"AdvaGen Pharma, Ltd"
112,Feb 13 2023 3:46PM,257292,20,Else Nutrition


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation,,,33
1,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation,,,50
2,Feb 14 2023 8:30AM,257297,10,ISDIN Corporation,,,5
3,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc.",,,6
4,Feb 14 2023 8:30AM,257303,10,"ClearSpec, LLC",,,2
5,Feb 14 2023 8:30AM,257294,10,"Snap Medical Industries, LLC",,,1
6,Feb 14 2023 8:30AM,257295,10,"Snap Medical Industries, LLC",,,3
7,Feb 14 2023 8:30AM,257304,10,Acute Outpatient Solutions,,,1
8,Feb 13 2023 3:54PM,257293,19,"AdvaGen Pharma, Ltd",,,11
9,Feb 13 2023 3:46PM,257292,20,Else Nutrition,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation,33,,
1,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation,50,,
2,Feb 14 2023 8:30AM,257297,10,ISDIN Corporation,5,,
3,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc.",6,,
4,Feb 14 2023 8:30AM,257303,10,"ClearSpec, LLC",2,,
5,Feb 14 2023 8:30AM,257294,10,"Snap Medical Industries, LLC",1,,
6,Feb 14 2023 8:30AM,257295,10,"Snap Medical Industries, LLC",3,,
7,Feb 14 2023 8:30AM,257304,10,Acute Outpatient Solutions,1,,
8,Feb 13 2023 3:54PM,257293,19,"AdvaGen Pharma, Ltd",11,,
9,Feb 13 2023 3:46PM,257292,20,Else Nutrition,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation,33,,
1,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation,50,,
2,Feb 14 2023 8:30AM,257297,10,ISDIN Corporation,5,,
3,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc.",6,,
4,Feb 14 2023 8:30AM,257303,10,"ClearSpec, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation,33.0,NaN,NaN
1,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation,50.0,NaN,NaN
2,Feb 14 2023 8:30AM,257297,10,ISDIN Corporation,5.0,NaN,NaN
3,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc.",6.0,NaN,NaN
4,Feb 14 2023 8:30AM,257303,10,"ClearSpec, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation,33.0,0.0,0.0
1,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation,50.0,0.0,0.0
2,Feb 14 2023 8:30AM,257297,10,ISDIN Corporation,5.0,0.0,0.0
3,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc.",6.0,0.0,0.0
4,Feb 14 2023 8:30AM,257303,10,"ClearSpec, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5916954,240.0,1.0,0.0
12,257253,0.0,1.0,0.0
19,2057969,20.0,2.0,63.0
20,771860,23.0,0.0,0.0
21,771776,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5916954,240.0,1.0,0.0
1,12,257253,0.0,1.0,0.0
2,19,2057969,20.0,2.0,63.0
3,20,771860,23.0,0.0,0.0
4,21,771776,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,240.0,1.0,0.0
1,12,0.0,1.0,0.0
2,19,20.0,2.0,63.0
3,20,23.0,0.0,0.0
4,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,240.0
1,12,Released,0.0
2,19,Released,20.0
3,20,Released,23.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,21
Status,,,,,
Completed,0.0,0.0,63.0,0.0,0.0
Executing,1.0,1.0,2.0,0.0,0.0
Released,240.0,0.0,20.0,23.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,21
0,Completed,0.0,0.0,63.0,0.0,0.0
1,Executing,1.0,1.0,2.0,0.0,0.0
2,Released,240.0,0.0,20.0,23.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,21
0,Completed,0.0,0.0,63.0,0.0,0.0
1,Executing,1.0,1.0,2.0,0.0,0.0
2,Released,240.0,0.0,20.0,23.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()